In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_excel('/content/drive/MyDrive/NNFL-Assignment2/data55.xlsx', header= None)
df[19] = df[19].replace({0 : -1, 1 : 1})
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,-1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,-1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,-1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,-1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,-1


In [4]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1, random_state = 7).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

208 19


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0123,0.0309,0.0169,0.0313,0.0358,0.0102,0.0182,0.0579,0.1122,0.0835,0.0548,0.0847,0.2026,0.2557,0.1870,0.2032,0.1463,0.2849,0.5824,-1
1,0.0374,0.0586,0.0628,0.0534,0.0255,0.1422,0.2072,0.2734,0.3070,0.2597,0.3483,0.3999,0.4574,0.5950,0.7924,0.8272,0.8087,0.8977,0.9828,1
2,0.0107,0.0453,0.0289,0.0713,0.1075,0.1019,0.1606,0.2119,0.3061,0.2936,0.3104,0.3431,0.2456,0.1887,0.1184,0.2080,0.2736,0.3274,0.2344,1
3,0.1313,0.2339,0.3059,0.4264,0.4010,0.1791,0.1853,0.0055,0.1929,0.2231,0.2907,0.2259,0.3136,0.3302,0.3660,0.3956,0.4386,0.4670,0.5255,1
4,0.1150,0.1163,0.0866,0.0358,0.0232,0.1267,0.2417,0.2661,0.4346,0.5378,0.3816,0.0991,0.0616,0.1795,0.3907,0.3602,0.3041,0.2428,0.4060,1


In [5]:
datan = df.values
X = datan[:, 0:19]

X = (X - np.min(X, axis = 0))/(np.max(X, axis=0)-np.min(X, axis=0))
#X = (X - X.mean(axis = 0))/(X.std(axis = 0))
y = datan[:, -1:]
print(X.shape, y.shape)

(208, 19) (208, 1)


In [6]:
num_classes = len(np.unique(y))
print(num_classes)

2


In [7]:
def activation(X_i, w, b):
  #X_i is 1 sample of X, so X_i.shape will be 1 x n
  #w.shape will be n x 1 
  return (np.sign(np.dot(X_i, w) + b))

In [8]:
def activation2(X_i, w, b):
  z = np.dot(X_i, w) + b
  return 1/(1 + np.exp(-z))

In [9]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(149, 19) (17, 19) (42, 19) (149, 1) (17, 1) (42, 1)


In [10]:
def update_parameters(X_i, y_i, w, b, alpha, o):
  # w.shape is n x 1
  # X_i.shape is 1 x n
  # y_i.shape is 1 x 1
  # b.shape us 1 x 1

  w = w + (alpha)*(np.dot(X_i.T, y_i - o))
  b = b + (alpha)*(y_i - o)
  return w, b

In [11]:
def perceptron(X, y, alpha, iterations):
  m = X.shape[0]
  n = X.shape[1]
  np.random.seed(3)
  w = np.random.randn(n, 1)
  b = 0
  mistakes_list = []
  min_mistakes = math.inf
  w_best, b_best = w, b
  for t in range(iterations):
    mistakes = 0
    for i in range(m):
      X_i = X[i:i+1]
      y_i = y[i:i+1]
      hypothesis = activation(X_i, w, b)
      if(hypothesis != y_i):
        mistakes = mistakes + 1
      w, b = update_parameters(X_i, y_i, w, b, alpha, hypothesis)
    if mistakes < min_mistakes:
      min_mistakes = mistakes
      w_best, b_best = w, b
    mistakes_list.append(mistakes)
  return w_best, b_best, mistakes_list

In [12]:
def perceptron2(X, y, alpha, iterations):
  m = X.shape[0]
  n = X.shape[1]
  np.random.seed(3)
  w = np.random.randn(n, 1)
  b = 0
  mistakes_list = []
  min_mistakes = math.inf
  w_best, b_best = w, b
  for t in range(iterations):
    mistakes = 0
    for i in range(m):
      X_i = X[i:i+1]
      y_i = y[i:i+1]
      hypothesis = activation2(X_i, w, b)
      prediction = -1
      if (hypothesis >= 0.5):
        prediction = 1
      if(prediction != y_i):
        mistakes = mistakes + 1
      w, b = update_parameters(X_i, y_i, w, b, alpha, prediction)
    if mistakes < min_mistakes:
      min_mistakes = mistakes
      w_best, b_best = w, b
    mistakes_list.append(mistakes)
  return w_best, b_best, mistakes_list

In [15]:
alpha_vals = np.linspace(0.01, 0.1, 15).tolist()
iter_vals = np.linspace(100,500,15,endpoint=False).astype(int).tolist()

In [16]:
acc_max = 0
alpha_best, iters_best, w_best, b_best = 0, 0, 0, 0
for i in alpha_vals:
  for j in iter_vals:
    w, b, mistakes_list = perceptron2(X_train, y_train, i, j)
    predictions = activation2(X_val, w, b)

    TP, TN, FP, FN = 0, 0, 0, 0
    for k in range(len(y_val)):
      if (predictions[k] < 0.5 and y_val[k] == -1):
        TN = TN + 1
      elif (predictions[k] >= 0.5 and y_val[k] == 1):
        TP = TP + 1
      elif (predictions[k] < 0.5 and y_val[k] == 1):
        FN = FN + 1
      elif (predictions[k] >= 0.5 and y_val[k] == -1):
        FP = FP + 1
    acc = (TP + TN)/(TP + TN + FN + FP)
    if acc > acc_max:
      acc_max = acc
      alpha_best = i
      iters_best = j
      w_best = w
      b_best = b


In [19]:
predictions = activation2(X_test, w_best, b_best)

TP, TN, FP, FN = 0, 0, 0, 0
for k in range(len(y_val)):
  if (predictions[k] < 0.5 and y_val[k] == -1):
    TN = TN + 1
  elif (predictions[k] >= 0.5 and y_val[k] == 1):
    TP = TP + 1
  elif (predictions[k] < 0.5 and y_val[k] == 1):
    FN = FN + 1
  elif (predictions[k] >= 0.5 and y_val[k] == -1):
    FP = FP + 1

In [22]:
SE = TP/(TP+FN)
SP = TN/(TN+FP)
Acc = (TP+TN)/(TP+TN+FP+FN)

In [24]:
print(SE)
print(SP)
print(Acc)

0.8
0.9285714285714286
0.9170588235294118
